## PROJECT 2

### imports

In [35]:
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd
import datetime
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import regex as re

### constants

In [36]:
url = 'https://www.airbnb.com/s/homes?refinement_paths[]=%2Fhomes&date_picker_type=flexible_dates&search_mode=flex_destinations_search'
tag_ctg = '&category_tag=Tag%'
root = 'https://www.airbnb.com'

tag_list = {'Beachfront':'3A789', 'Shipping containers':'3A8157', 'Cabins':'3A5348','Yurts':'3A8192','Amazing pools':'3A677',
            'Domes':'3A8173', 'Cycladic Houses':'3A8227', 'A-frames':'3A8148', 'Islands':'3A675', 'Campers':'3A8166',
            'Tiny homes':'3A8186', 'Farms':'3A8175', 'Houseboats':'3A8176', 'Castles':'3A8047', 'Barns':'3A8159', 
            'Boats':'3A1073', 'Ryokans':'3A8228', 'Treehouses':'3A8188', 'Kezhan':'3A8231', 'Trulli':'3A8229'}

c = pd.read_csv(r"C:\Users\elaaf\Desktop\SDS\Project_2\Data\list_of_cities_of_saudi_arabia-1648j.csv")
cities = c['City name']


In [37]:
amenities_not_included = ["Bathroom","Bedroom and laundry","Essentials","Towels, bed sheets, soap, and toilet paper",
                           "Entertainment","Family","Heating and cooling","Home safety","Internet and office"
                           "Available throughout the listing","A desk or table with a chair and space for a laptop."
                           "Kitchen and dining","Bowls, chopsticks, plates, cups, etc.","Location features"
                           "Separate street or building entrance","Outdoor","Parking and facilities","Services"
                           "Not included","Bowls, chopsticks, plates, cups, etc.","Separate street or building entrance",
                           "An open space on the property usually covered in grass","Breakfast is provided","Allow stay for 28 days or more",
                           "Someone is available 24 hours a day to let guests in","This place may not have a smoke detector. Reach out to the host with any questions.",
                           "This place may not have a carbon monoxide detector. Reach out to the host with any questions."]

### functions

In [38]:
## set up firefox driver
DRIVER_PATH = 'C:\Program Files\Mozilla Firefox\geckodriver-v0.30.0-win64/geckodriver.exe'
def driver_set_up_firefox():
    geckodriver = "C:\Program Files\Mozilla Firefox\geckodriver-v0.30.0-win64" # path to the chromedriver executable
    os.environ["webdriver.firefox.driver"] = geckodriver
    
    
    
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference('permissions.default.image', 2)
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

    driver = webdriver.Firefox(executable_path=DRIVER_PATH, firefox_profile=firefox_profile)
    
    
    return driver

In [39]:
def open_city_page(city, driver):
    
    driver.find_element_by_css_selector('button._b2fxuo:nth-child(2)').click()
    #time.sleep(4)
    
    search_box = driver.find_element_by_xpath('//*[@id="bigsearch-query-attached-location-input"]')

    
    search_box.clear()
    
    search_box.send_keys(city+', Saudi Arabia')
    
    search_box.send_keys(Keys.RETURN)  

    time.sleep(3)
    
    search_button = driver.find_element_by_css_selector('._1mzhry13')

    search_button.click()
    


In [40]:
def get_pages_links(soup):
    link = soup.find_all('a', class_ ='_1y623pm')
    
    if link != []:
        last_link=link[-1].get('href')
        le = last_link.find('items_offset=')
        txt = last_link[le:]

        start = txt.find('=') + le + 1
        end = txt.find('&') + le
        max_offset = last_link[start:end]
        
        L = [root+last_link]
        
        i=0
        
        while (i!=int(max_offset)):
            L.append(root+last_link[:start]+str(i)+'&section_offset=2')
            i+=20
           
        return L
        
    return []

In [41]:
## find all links for all houses 

def find_links_of_a_page(soup):
    links_a = soup.find_all('a', class_='_mm360j')
    links = []
    for l in links_a:
        links.append(root + l.get('href'))
    return links

In [42]:
def get_links():
    driver = driver_set_up_firefox()
    url = 'https://www.airbnb.com/s/Rabigh-Saudi-Arabia/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=october&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&source=structured_search_input_header&search_type=user_map_move&click_referer=t%3ASEE_ALL%7Csid%3A7a124ac4-292c-45d6-87f1-de36f25a8936%7Cst%3ASTAYS_LARGE_AREA_DESTINATION_CAROUSELS&tab_id=home_tab&query=Saudi%20Arabia&ne_lat=22.872738901828196&ne_lng=39.99952439880374&sw_lat=22.41265888730569&sw_lng=38.59464768981937&zoom=10&search_by_map=true'
    driver.get(url)
    time.sleep(2)
    
    for city in cities:
        print(city)
        open_city_page(city, driver)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links = get_pages_links(soup)
        
        for l in links:
            driver.get(l)
            time.sleep(3)
            soup =  BeautifulSoup(driver.page_source, 'html.parser')
            pages = find_links_of_a_page(soup)
            for p in pages:
                tab_links.append(p)
        
    
    driver.close()
    #return tab_links
    
    

### Total links code

In [36]:
tab_links = [] 


get_links()

Al-Abwa
Al Artaweeiyah
Badr
Baljurashi
Bisha
Bareg
Buraydah
Al Bahah
Buq a
Dammam
Dhahran
Dhurma
Dahaban
Diriyah
Duba
Dumat Al-Jandal
Dawadmi
Farasan city
Gatgat
Gerrha
Gurayat
Al-Gwei'iyyah
Hautat Sudair
Habala
Hajrah
Haql
Al-Hareeq
Harmah
Ha'il
Hotat Bani Tamim
Hofuf
Huraymila
Hafr Al-Batin
Jabal Umm al Ru'us
Jalajil
Al Jawf
Jeddah
Jizan
Jizan Economic City
Jubail
Al Jafer
Khafji
Khaybar
King Abdullah Economic City
Khamis Mushayt
Al Kharj
Knowledge Economic City , Medina
Khobar
Al-Khutt
Layla
Lihyan
Al Lith
Al Majma'ah
Mastoorah
Al Mikhwah
Al-Mubarraz
Al Mawain
Mecca
Medina
Muzahmiyya
Najran
Al-Namas
Omloj
Al-Omran
Al-Oyoon
Qadeimah
Qatif
Qaisumah
Al Qunfudhah
Rabigh
Rafha
Ar Rass
Ras Tanura
Riyadh
Riyadh Al-Khabra
Rumailah
Sabt Al Alaya
Saihat
Safwa city
Sakakah
Sharurah
Shaqraa
Shaybah
As Sulayyil
Taif
Tabuk
Tanomah
Tarout
Tayma
Thadiq
Thuwal
Thuqbah
Turaif
Tabarjal
Udhailiyah
Al-`Ula
Um Al-Sahek
Unaizah
Uqair
'Uyayna
Uyun AlJiwa
Wadi Al-Dawasir
Al Wajh
Yanbu
Az Zaimah
Zulfi


In [83]:
links_df = pd.DataFrame(tab_links)

In [34]:
#pd.to_pickle(links_df, 'links_.csv')
tab_links = pd.read_pickle('links_.csv').iloc[:, 0] 

In [18]:
df.shape

(4972, 579)

### Data Collection

In [288]:
df = pd.DataFrame(columns=['rating', 'house_count', 'city',
                          'province', 'country', 'superhost',
                          'price', 'start_date', 'end_date', 'house_type'])




In [414]:


driver = driver_set_up_firefox()

L = []

cols = ['rating', 'house_rating_count', 'city',
        'province', 'country', 'superhost',
        'price', 'start_date', 'end_date', 'house_type', 'link']


for link in tab_links[4963:]:
    
    driver.get(link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        city, prov, country = get_city_prov_country(soup)
        
    except:
        continue
    start_date, end_date = get_dates(soup)
    
    L = [get_rating(soup),
        get_rating_count(soup),
        city,
        prov, 
        country,
        get_superhost(soup),
        get_price(soup),
        start_date,
        end_date,
        get_house_type(soup),
        link]
    
    d = {}
    
    
    for e, r in zip(L, cols):
        d[r] = e
        #print(r, e)
    
    d.update(get_rooms_details(soup))  
    d.update(get_amen(soup))
    
    DF = pd.DataFrame([d])
    
    df = pd.concat([df, DF])
        
driver.close()

KeyboardInterrupt: 

In [389]:
x = df['link'].iloc[-1]

In [415]:
df.shape

(4972, 580)

In [391]:
tab_links[tab_links == x]


4963    https://www.airbnb.com/rooms/52457484?previous...
Name: 0, dtype: object

In [372]:
tab_links[2084]


'https://www.airbnb.com/rooms/34847655?previous_page_section_name=1000&translate_ugc=false&federated_search_id=48e1dc98-3a3e-4439-9067-9ef60a10fddb'

In [19]:
df.to_csv('uncleaned_airbnb.csv')

### Data collection functions 

In [274]:
def get_amen(soup):
    d = {}
    time.sleep(3)
    amenities_button = driver.find_element_by_css_selector(".b1sec48q")
    amenities_button.click()
    
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_amenities = soup.find_all("div", class_ = "_gw4xx4")
    
    
    
    Unav_amenities = []
    av_amenities = []
    round_num = 1
    for x in list_amenities:
        if("Unavailable" in x.getText() and x.getText() not in amenities_not_included):
            y = x.getText(separator=" ")
            y = y.replace("Unavailable: ","")
            z = y.split()
            j = " ".join(sorted(set(z), key=z.index))
            Unav_amenities.append(j)
            d[j] = 0
        elif("Unavailable" not in x.getText() and x.getText() not in amenities_not_included):
            av_amenities.append(x.getText())
            d[x.getText()] = 1
        
    return d
            




In [ ]:
def get_house_type(soup):
    host_Type = soup.find('div', class_='_88xxct').find("h2")
    house_text = host_Type.text
    index = int(house_text.index(" hosted"))
    return house_text[0:index]
    

In [8]:
## find house type and links 

def get_houses_types_and_urls(soup):
    house_types = set(soup.find_all('div', {'class' : "_j1kt73"}))
    house_types = set([i.text for i in house_types if i.text != 'More'])
    links = []
    for h in house_types:
        links.append(url+tag_ctg+tag_list[h])
    return house_types, links

In [9]:
## convert the price to an integer

def price_to_float(moneystring):
    moneystring = moneystring.replace('SR', '').replace(',', '')
    return float(moneystring)

In [10]:
## convert the str to an date

def str_to_date(string):
    return datetime.datetime.strptime(string, '%m/%d/%Y').date()

In [258]:
def get_rating(soup):
    rating = soup.find('span', class_='_1ne5r4rt')
    if rating != None and rating.text != '':
        return float(rating.text)
    else:
        return np.nan

In [296]:
def get_city_prov_country(soup):
    condition = soup.find('span', class_ = '_pbq7fmm')
    if condition != None:
        x = condition.text.split(',')
        if len(x) < 2:
            return 'Saudi Arabia', 'Saudi Arabia', 'Saudi Arabia'
        elif len(x) == 2:
            l = get_city_country(soup)
            return l[0].strip(), l[0].strip(), l[1].strip()
        
        return x[0].strip(), x[1].strip(), x[2].strip()

    return None, None, None

In [16]:
def get_superhost(soup):
    condition = soup.find('span', class_='_63km3vu')
    superhost = False
    if condition != None:
        superhost = True
        
    return superhost


In [17]:
def get_price(soup):
    
    s = soup.find('span', class_='_tyxjp1')
    if s != None:
        return price_to_float(s.text)
    else:
        return np.nan


In [18]:
def get_dates(soup):
    start_date = soup.find('div', {'class':'_1g8031c', 'data-testid':'change-dates-checkIn'})
    end_date = soup.find('div', {'class':'_1g8031c', 'data-testid':'change-dates-checkOut'})
    if start_date != None and end_date != None:
        return str_to_date(start_date.text), str_to_date(end_date.text)
    else:
        return None, None

In [307]:
def get_rooms_details(soup):
    temp = soup.find('div', {'class':'_tqmy57'})
    allowed = ['bed', 'beds','guests', 'guest',
              'bath', 'baths', ]
    dictionary = {}
    if temp != None:
        temp = temp.find_all('span')
        details = list(set([x.text.strip(' ,·') for x in temp]))
        #print(details)
        details.remove('')
        for d in details:
            t = d.split(' ')
            if 'shared' in t or 'Shared' in t:
                continue
            if len(t) == 2:
                dictionary[t[1]] = float(t[0])
            else:
                continue
    return dictionary

In [21]:
def build_urls(url, listings_per_page=20, pages_per_location=15):
    """Builds links for all search pages for a given location"""
    
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = url + f'&items_offset={offset}'
        url_list.append(url_pagination)
       
    return url_list

## additional info

In [454]:
test = 'https://www.airbnb.com/rooms/45166897?translate_ugc=false&federated_search_id=db739068-e5b9-4c46-a375-a0b1d73c93ec&source_impression_id=p3_1633966265_Ee3sXEKNjeTAXQHF&guests=1&adults=1'

In [455]:


driver = driver_set_up_firefox()
driver.get(test)
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.close()

In [458]:
def get_rating_count(soup):
    rating_count = soup.find('span', class_='_1qx9l5ba')
    if rating_count != None:
        return rating_count.text.replace(' reviews', '').strip('()')
    else:
        return 0

In [459]:
get_rating_count(soup)

'40'

In [209]:
df = pd.read_csv(r'C:\Users\elaaf\Desktop\SDS\Project_2\Data Cleaning\cleaned_v6.csv', index_col=0)

In [23]:
df.head(2)

,rating,city,province,superhost,price,house_type,link,Hangers,Bed linens,Extra pillows and blankets,...,game console,guest_num,bath_num,id,pool,backyard,free carport,ski-in,beach access,balcony
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,buraydah,6,0.0,1100,Earth house,https://www.airbnb.com/rooms/44085675?previous...,1,1,1,...,1,10,3.0,0.0BuraydahAlQassimFalse1100.0Earth house,0,1,0,0,0,1
1,0.0,buraydah,6,0.0,2200,Entire chalet,https://www.airbnb.com/rooms/19854001?previous...,1,0,0,...,0,4,4.0,0.0BuraydahAlQassimFalse2200.0Entire chalet,1,0,0,0,0,0


In [211]:
df_test = df.copy(deep=True)

In [162]:
links_of_rows_with_ratings = df_test[df_test['rating']!=0]['link']

In [163]:
links_of_rows_with_ratings[:5]

index
11    https://www.airbnb.com/rooms/38629962?previous...
13    https://www.airbnb.com/rooms/49255433?previous...
17    https://www.airbnb.com/rooms/43965096?previous...
20    https://www.airbnb.com/rooms/42675643?previous...
29    https://www.airbnb.com/rooms/42653518?previous...
Name: link, dtype: object

In [48]:
def get_rating_count(soup):
    rating_count = soup.find('span', class_='_1qx9l5ba')
    if rating_count != None:
        return rating_count.text.replace(' reviews', '').strip('()')
    else:
        return None

In [164]:
df_rating_count = pd.DataFrame(columns=['i'])

In [165]:
df_rating_count['i']= links_of_rows_with_ratings.index

In [87]:
df_rating_count = df_rating_count.set_index('index')

In [71]:
driver = driver_set_up_firefox()
L = []

for link in links_of_rows_with_ratings:
    driver.get(link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    L.append(get_rating_count(soup))
    #df_rating_count[]
    

driver.close()

In [88]:
len(L) 

799

In [166]:
df_rating_count

,i
0,11
1,13
2,17
3,20
4,29
...,...
794,1850
795,1853
796,1854
797,1856


In [177]:
links_of_rows_with_ratings

index
11      https://www.airbnb.com/rooms/38629962?previous...
13      https://www.airbnb.com/rooms/49255433?previous...
17      https://www.airbnb.com/rooms/43965096?previous...
20      https://www.airbnb.com/rooms/42675643?previous...
29      https://www.airbnb.com/rooms/42653518?previous...
                              ...                        
1850    https://www.airbnb.com/rooms/50945676?previous...
1853    https://www.airbnb.com/rooms/49294137?previous...
1854    https://www.airbnb.com/rooms/52126578?previous...
1856    https://www.airbnb.com/rooms/48752150?previous...
1857    https://www.airbnb.com/rooms/52075274?previous...
Name: link, Length: 799, dtype: object

In [168]:
df_rating = pd.DataFrame(L, columns=['rating'])

In [191]:
d = df_rating.set_index(keys=df_rating_count['i'])

In [207]:
count = d.join(links_of_rows_with_ratings, d.index)

In [208]:
count

,rating,link
i,,
11,4,https://www.airbnb.com/rooms/38629962?previous...
13,4,https://www.airbnb.com/rooms/49255433?previous...
17,5,https://www.airbnb.com/rooms/43965096?previous...
20,5,https://www.airbnb.com/rooms/42675643?previous...
29,3,https://www.airbnb.com/rooms/42653518?previous...
...,...,...
1850,50,https://www.airbnb.com/rooms/50945676?previous...
1853,56,https://www.airbnb.com/rooms/49294137?previous...
1854,16,https://www.airbnb.com/rooms/52126578?previous...


In [212]:
df_test.head()

,rating,city,province,superhost,price,house_type,link,Hangers,Bed linens,Extra pillows and blankets,...,game console,guest_num,bath_num,id,pool,backyard,free carport,ski-in,beach access,balcony
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,buraydah,6,0.0,1100,Earth house,https://www.airbnb.com/rooms/44085675?previous...,1,1,1,...,1,10,3.0,0.0BuraydahAlQassimFalse1100.0Earth house,0,1,0,0,0,1
1,0.0,buraydah,6,0.0,2200,Entire chalet,https://www.airbnb.com/rooms/19854001?previous...,1,0,0,...,0,4,4.0,0.0BuraydahAlQassimFalse2200.0Entire chalet,1,0,0,0,0,0
2,0.0,buraydah,6,0.0,400,Shared room in chalet,https://www.airbnb.com/rooms/37763947?previous...,1,0,0,...,0,16,4.5,0.0BuraydahAlQassimFalse400.0Shared room in ch...,1,0,0,0,0,0
3,0.0,buraydah,6,0.0,550,Private room in resort,https://www.airbnb.com/rooms/44031535?previous...,1,1,1,...,0,5,2.0,0.0BuraydahAlQassimFalse550.0Private room in r...,1,0,0,0,0,0
4,0.0,buraydah,6,0.0,2500,Entire villa,https://www.airbnb.com/rooms/35754885?previous...,1,0,0,...,0,7,3.0,0.0BuraydahAlQassimFalse2500.0Entire villa,1,0,0,0,0,0


i
11       4
13       4
17       5
20       5
29       3
        ..
1850    50
1853    56
1854    16
1856    25
1857     3
Name: rating, Length: 799, dtype: object

In [297]:
df_test = df.copy(deep=True)
df['rating_count'] = 0

In [245]:
#df_test.loc[d.index      ,'rating_count'] = d['rating']

In [260]:
df_test.iloc[1856]['rating_count'] 

0

In [298]:
L = d.index
error_index = []
for i in L:
    try:
        df.loc[i,'rating_count'] = int(d.loc[i,'rating'])
    except:
        error_index.append(i)
        continue


In [299]:
error_index

[882, 1627]

In [300]:
df.to_csv('data_v6_with_ratingcount.csv',index=0)

In [302]:
c = list(df.columns)

In [304]:
#c